In [ ]:
#importing libraries for data creation
import random
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle as pk
import numba as nb

In [ ]:
#Random lattice creation
def latticecreate(n):
    lattice = np.random.choice(a=[-1,1],size=(n,n))
    return lattice

In [ ]:
#metropolis algorithm
@nb.jit
def calculateenergy(lattice,n,j,T):
    T=float(T)
    mag=0
    mo=0
    meg=0
    for k in range (0,1000):
        for l in range (0,100):
            x=np.random.randint(0,n)
            y=np.random.randint(0,n)
            
            nb = lattice[(x+1)%n,y] + lattice[x,(y+1)%n] + lattice[(x-1)%n,y] + lattice[x,(y-1)%n]
            
            deltaE=lattice[x,y]*2*nb*j
            p=random.uniform(0,1)
            if deltaE<0:
                lattice[x,y]=lattice[x,y]*(-1)
                mag1=magnetisation(lattice,n)
                meg=meg+mag1**2
                mag=mag+mag1**4
            else:
                r=math.exp((-deltaE)/(T))
                if p<r:
                    lattice[x,y]=lattice[x,y]*(-1)
                    meg=meg+mag1**2
                    mag=mag+mag1**4
                else:
                    lattice[x,y]=lattice[x,y]
                    mag1=magnetisation(lattice,n)
                    meg=meg+mag1**2
                    mag=mag+mag1**4
            mo=mo+1
            
       
    meg=(meg/mo)*3
    mag=(mag/mo)
            
    return (1-(mag/(meg**2)))

In [ ]:
#magntisation calculation function
@nb.jit(nopython=True)
def magnetisation(lattice,n):
    mag1=0
    for i in range(0,n):
        for j in range(0,n):
            mag1=mag1+(lattice[i,j])
            
    return (mag1/(n**2))

In [ ]:
def maglist(n,T,j):
    lattice1=latticecreate(n)
    cur=calculateenergy(lattice1,n,j,T)
    return cur

In [ ]:
#the function cullects and apends binders cumulant values corresponding to temperature and latttice no.
def curietemp(j):
    cur_y=[]
    Temp_x=[]
    nlist_x=[]
    n=5
    while n<60:
        cur=[]
        Temp=[]
        nlist=[]
        T=0.2
        while T<5.1:
            if T<2:
                cur.append(maglist(n,T,j))
                Temp.append(T)
                T=T+0.02
            elif 2<T<3:
                cur.append(maglist(n,T,j))
                Temp.append(T)
                T=T+0.001
            else:
                cur.append(maglist(n,T,j))
                Temp.append(T)
                T=T+0.1
            
            nlist.append(n)
        cur_y=cur_y+cur
        nlist_x=nlist_x+nlist
        Temp_x=Temp_x+Temp
        n=n+2
    return cur_y,Temp_x,nlist_x

In [ ]:
j=1
Train_y,Train_x1,Train_x2=curietemp(j)

In [ ]:
#data reshaping 
Train_X=np.array([[Train_x1],[Train_x2]])
Train_Y=np.array([[Train_y]])
Train_Y=Train_Y.reshape(1,len(Train_x1))
Train_X=Train_X.reshape(2,len(Train_x1))
Train_X=np.transpose(Train_X)
Train_Y=np.transpose(Train_Y)

In [ ]:
#importing libraries for training model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Flatten,PReLU,Dropout,LeakyReLU,LSTM
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor

In [ ]:
#Defing model with ,3 dense layers and one dropout layer to avoid over fitting
model = Sequential()                                               #model creation and addition of layers
model.add(Dense(1100, input_dim=2, kernel_initializer='normal', activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()

In [ ]:
#compiling model
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [ ]:
#Training the model
history = model.fit(Train_X, Train_Y, epochs=50, batch_size=100,  verbose=1, validation_split=0.1)

In [ ]:
print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#saving the model for future prediction
model.save('Filename.h5')